In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

employee_status=pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
employee_status.head()


In [ ]:
print(employee_status.shape)
print(employee_status.columns)
print(employee_status.dtypes)

In [ ]:
data=employee_status.copy()
for col in data.columns:
    if data[col].dtype in ['float64', 'int64']:
        data[col] = data[col].fillna(data[col].median())
    else:
        data[col] = data[col].fillna(data[col].mode()[0])


In [ ]:
data.to_csv("clean_employee_attrition.csv", index=False)

In [ ]:
data.drop(columns=['Over18'])

In [ ]:
from sklearn.preprocessing import StandardScaler

num_cols = ["Age", "DailyRate", "DistanceFromHome", "HourlyRate", "MonthlyIncome", "MonthlyRate", "NumCompaniesWorked",
            "PercentSalaryHike", "TotalWorkingYears", "TrainingTimesLastYear", "YearsAtCompany", "YearsInCurrentRole",
            "YearsSinceLastPromotion"]

scaler = StandardScaler()
data[num_cols] = scaler.fit_transform(data[num_cols])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import seaborn as sb
for i in data.select_dtypes(include=['object']).columns:
    data[i] = le.fit_transform(data[i])
corellation = data.corr()['Attrition'].sort_values(ascending=False)
print(corellation)
sb.heatmap(corellation.to_frame(), annot=False, cmap='coolwarm')

In [ ]:
import matplotlib.pyplot as plt
sb.countplot(x='Attrition', data=data)
plt.title('Employee Attrition Count')
plt.show()


In [ ]:
features=data.drop('Attrition', axis=1)
target=data['Attrition']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train,y_train)


In [ ]:
import pickle
with open('employee_attrition_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sb
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
sb.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()